In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import re
from collections import Counter


In [93]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)


In [94]:
df = pd.read_csv("Queries.csv")
df.head()

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,35.83%,1.61
1,thecleverprogrammer,2809,3456,81.28%,1.02
2,python projects with source code,2077,73380,2.83%,5.94
3,classification report in machine learning,2012,4959,40.57%,1.28
4,the clever programmer,1931,2528,76.38%,1.09


In [95]:
df.shape

(1000, 5)

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Top queries  1000 non-null   object 
 1   Clicks       1000 non-null   int64  
 2   Impressions  1000 non-null   int64  
 3   CTR          1000 non-null   object 
 4   Position     1000 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB


In [97]:
df.isna().sum()

Top queries    0
Clicks         0
Impressions    0
CTR            0
Position       0
dtype: int64

In [98]:
# Preprocessing
df['CTR'] = df['CTR'].str.rstrip('%').astype('float') / 100

In [99]:
df.describe()

,Clicks,Impressions,CTR,Position
count,1000.0000,1000.000000,1000.000000,1000.000000
mean,172.2750,1939.466000,0.225726,3.985930
std,281.0221,4856.702605,0.165270,2.841842
min,48.0000,62.000000,0.002900,1.000000
25%,64.0000,311.000000,0.080750,2.010000
50%,94.0000,590.500000,0.197150,3.120000
75%,169.0000,1582.750000,0.340250,5.342500
max,5223.0000,73380.000000,0.854800,28.520000


**Insights**

- Out of 1,000, on an average 172 clicks.
- Out of 1,000, on an average 1,939 impressions.
- Out of 1,000, on an average 0.225 (22.5%) Click-Through Rate.
- The Impressions column shows a massive gap between the average (1,939) and the maximum (73,380), indicating that a few - items are getting significantly more visibility than others.
- The median (50%) for clicks is only 94, which is much lower than the mean of 172, suggesting that the average is being pulled up by a small number of very high-performing entries.
- While the average position is roughly 4, some items have ranked as high as 1 and as low as 28.5.

In [100]:
def clean_and_split(query):
    words = re.findall(r'\b[a-zA-Z]+\b', query.lower())
    return words

word_counts = Counter()
for query in df['Top queries']:
    word_counts.update(clean_and_split(query))

word_freq_df = pd.DataFrame(word_counts.most_common(20), columns=['Word', 'Frequency'])

fig = px.bar(word_freq_df, x='Word', y='Frequency', title='Top 20 Most Common Words in Search Queries',width=1000, height=800)
fig.show()

In [101]:
# Top queries by Clicks and Impressions
top_queries_clicks_vis = df.nlargest(10, 'Clicks')[['Top queries', 'Clicks']]
top_queries_impressions_vis = df.nlargest(10, 'Impressions')[['Top queries', 'Impressions']]

fig_clicks = px.bar(top_queries_clicks_vis, x='Top queries', y='Clicks', title='Top Queries by Clicks',width=1000, height=800)
fig_impressions = px.bar(top_queries_impressions_vis, x='Top queries', y='Impressions', title='Top Queries by Impressions',width=1000, height=800)
fig_clicks.show()
fig_impressions.show()

In [ ]:
# Queries with highest and lowest CTR
top_ctr_vis = df.nlargest(10, 'CTR')[['Top queries', 'CTR']]
bottom_ctr_vis = df.nsmallest(10, 'CTR')[['Top queries', 'CTR']]
fig_top_ctr = px.bar(top_ctr_vis, x='Top queries', y='CTR', title='Top Queries by CTR',width=1000, height=800)
fig_bottom_ctr = px.bar(bottom_ctr_vis, x='Top queries', y='CTR', title='Bottom Queries by CTR',width=1000, height=800)
fig_top_ctr.show()
fig_bottom_ctr.show()

In [103]:
# Correlation matrix visualization
correlation_matrix = df[['Clicks', 'Impressions', 'CTR', 'Position']].corr()
fig_corr = px.imshow(correlation_matrix, text_auto=True, title='Correlation Matrix')
fig_corr.show()

In this correlation matrix:

- Clicks and Impressions are positively correlated, meaning more Impressions tend to lead to more Clicks.

- Clicks and CTR have a weak positive correlation, implying that more Clicks might slightly increase the Click-Through Rate.

- Clicks and Position are weakly negatively correlated, suggesting that higher ad or page Positions may result in fewer Clicks.

- Impressions and CTR are negatively correlated, indicating that higher 

- Impressions tend to result in a lower Click-Through Rate.

- Impressions and Position are positively correlated, indicating that ads or pages in higher Positions receive more Impressions.

- CTR and Position have a strong negative correlation, meaning that higher Positions result in lower Click-Through Rates.

Detecting Anomalies in Search Queries

- Now, let’s detect anomalies in search queries. You can use various techniques 
for anomaly detection. A simple and effective method is the Isolation Forest algorithm, which works well with different data distributions and is efficient with large datasets

In [104]:
from sklearn.ensemble import IsolationForest
features = df[['Clicks', 'Impressions', 'CTR', 'Position']]
iso_forest = IsolationForest(n_estimators=100, contamination=0.01)  
iso_forest.fit(features)
df['anomaly'] = iso_forest.predict(features)
anomalies = df[df['anomaly'] == -1]

In [105]:
print(anomalies[['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position']])

                          Top queries  Clicks  Impressions     CTR  Position
0         number guessing game python    5223        14578  0.3583      1.61
1                 thecleverprogrammer    2809         3456  0.8128      1.02
2    python projects with source code    2077        73380  0.0283      5.94
4               the clever programmer    1931         2528  0.7638      1.09
15         rock paper scissors python    1111        35824  0.0310      7.19
21              classification report     933        39896  0.0234      7.53
34           machine learning roadmap     708        42715  0.0166      8.97
82                           r2 score     367        56322  0.0065      9.33
167               text to handwriting     222        11283  0.0197     28.52
929                     python turtle      52        18228  0.0029     18.75


- The anomalies in our search query data are not just outliers. They are indicators of potential areas for growth, optimization, and strategic focus. These anomalies are reflecting emerging trends or areas of growing interest. Staying responsive to these trends will help in maintaining and growing the website’s relevance and user engagement.

- Search Queries Anomaly Detection means identifying queries that are outliers according to their performance metrics. It is valuable for businesses to spot potential issues or opportunities, such as unexpectedly high or low CTRs.